# Parte 2: Uso de consultas SQL

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oramosul/abd-files/blob/main/spark/2-spark-sql/2-Consultas-SQL.ipynb)


La documentación sobre el API de SQL en Spark es: https://spark.apache.org/docs/latest/sql-ref.html

In [ ]:
!pip install -q pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [18]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/personas.csv
!wget -q https://raw.githubusercontent.com/oramosul/abd-files/main/spark/datos/poema.txt

## Ejemplo 1: A partir de un DataFrame

**Creación de un DataFrame**

In [5]:
df = spark.read.load("personas.csv",
                     format="csv", sep=";", inferSchema=True, header=True)
df.show(2)

+------+----+-------------+
|nombre|edad|      trabajo|
+------+----+-------------+
| Jorge|  30|Desarrollador|
| María|  23|    Psicóloga|
+------+----+-------------+
only showing top 2 rows



**Vista: "Creación de una tabla"**

Para poder realizar consultas ("queries") usando SQL sobre un DataFrame, primero se debe crear una vista temporal del DataFrame usando `createOrReplaceTempView`. Luego sobre esta vista temporal se realiza las consultas usando SQL.

In [6]:
# Crear una "tabla" llamada "tabla1" con los datos del dataframe df
df.createOrReplaceTempView('tabla1')

**Ejemplos de consultas:**

Se puede realizar consultas de tipo SQL sobre la vista temporal creada, que en este caso se llama `tabla1`. El resultado es un DataFrame de PySpark.

In [7]:
spark.sql("SELECT * FROM tabla1")

DataFrame[nombre: string, edad: int, trabajo: string]

Para visualizar el resultado se puede colocar directamente `show`, o se puede almacenar en una variable (DataFrame) y visualizar dicho DataFrame usando `show`.

In [8]:
df2 = spark.sql("SELECT * FROM tabla1")
df2.show()

+--------+----+--------------+
|  nombre|edad|       trabajo|
+--------+----+--------------+
|   Jorge|  30| Desarrollador|
|   María|  23|     Psicóloga|
|   Pedro|  25|       Abogado|
|Jennifer|  21|Administradora|
| Mariana|  23|    Influencer|
|   Adalí|  24|     Estilista|
+--------+----+--------------+



In [9]:
spark.sql("SELECT * FROM tabla1 ORDER BY edad DESC LIMIT 2").show()

+------+----+-------------+
|nombre|edad|      trabajo|
+------+----+-------------+
| Jorge|  30|Desarrollador|
| Pedro|  25|      Abogado|
+------+----+-------------+



In [10]:
spark.sql("SELECT * FROM tabla1 WHERE edad<26").show()

+--------+----+--------------+
|  nombre|edad|       trabajo|
+--------+----+--------------+
|   María|  23|     Psicóloga|
|   Pedro|  25|       Abogado|
|Jennifer|  21|Administradora|
| Mariana|  23|    Influencer|
|   Adalí|  24|     Estilista|
+--------+----+--------------+



In [11]:
spark.sql("SELECT * FROM tabla1 WHERE nombre LIKE 'Jorge'").show()

+------+----+-------------+
|nombre|edad|      trabajo|
+------+----+-------------+
| Jorge|  30|Desarrollador|
+------+----+-------------+



In [12]:
spark.sql("SELECT count(*) AS Elementos from tabla1").show()

+---------+
|Elementos|
+---------+
|        6|
+---------+



**Creación de UDF (User Defined Function)**

In [13]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

In [14]:
@pandas_udf("integer")
def suma_cien(s: pd.Series) -> pd.Series:
    return s + 100

spark.udf.register("suma_cien", suma_cien)

Uso del UDF para una consulta sobre los datos usando SQL.

In [15]:
spark.sql("SELECT nombre, suma_cien(edad) AS edad100 FROM tabla1").show()

+--------+-------+
|  nombre|edad100|
+--------+-------+
|   Jorge|    130|
|   María|    123|
|   Pedro|    125|
|Jennifer|    121|
| Mariana|    123|
|   Adalí|    124|
+--------+-------+



In [16]:
from pyspark.sql.functions import expr

df.selectExpr('suma_cien(edad)').show()

+---------------+
|suma_cien(edad)|
+---------------+
|            130|
|            123|
|            125|
|            121|
|            123|
|            124|
+---------------+



## Ejemplo 2: A partir de un RDD

Si la entrada de datos utiliza un RDD, primero se debe convertir a un DataFrame y luego recién a una vista temporal para poder realizar consultas SQL sobre dicha vista temporal.

In [19]:
sc = spark.sparkContext

# Creación de un RDD a partir de un archivo de texto
rdd = sc.textFile("poema.txt")
rdd.take(3)

['Puedo escribir los versos más tristes esta noche.',
 '',
 'Escribir, por ejemplo: “La noche está estrellada,']

In [20]:
# Se importa re para usar expresiones regulares
import re

# Contar palabras usando el esquema MapReduce
rdd2 = rdd.flatMap(lambda x: x.lower().split())\
          .map(lambda x: (re.sub(r'\W+', '', x), 1)) \
          .reduceByKey(lambda x,y: x+y)

rdd2.take(10)

[('los', 8),
 ('tristes', 3),
 ('ejemplo', 1),
 ('estrellada', 2),
 ('y', 8),
 ('tiritan', 1),
 ('azules', 1),
 ('astros', 1),
 ('lo', 3),
 ('lejos', 3)]

In [21]:
from pyspark.sql import Row

# Crear un RDD con filas de tipo "Row"
rdd3 = rdd2.map(lambda x: Row(palabra=x[0], contador=x[1], longitud=len(x[0])))
rdd3.take(3)

[Row(palabra='los', contador=8, longitud=3),
 Row(palabra='tristes', contador=3, longitud=7),
 Row(palabra='ejemplo', contador=1, longitud=7)]

In [22]:
# Crear un DataFrame a partir del RDD
df = spark.createDataFrame(rdd3)

df.show(4)

+----------+--------+--------+
|   palabra|contador|longitud|
+----------+--------+--------+
|       los|       8|       3|
|   tristes|       3|       7|
|   ejemplo|       1|       7|
|estrellada|       2|      10|
+----------+--------+--------+
only showing top 4 rows



In [23]:
# Crear una vista temporal
df.createOrReplaceTempView("tabla2")

# Consultas SQL en la tabla
spark.sql("SELECT palabra, longitud FROM tabla2 WHERE contador>=3 ORDER BY longitud DESC").show(10)

+--------+--------+
| palabra|longitud|
+--------+--------+
|escribir|       8|
| tristes|       7|
| perdido|       7|
|  versos|       6|
|  quiero|       6|
|   lejos|       5|
|   puedo|       5|
|   noche|       5|
|   veces|       5|
|    como|       4|
+--------+--------+
only showing top 10 rows

